# Diversity measures from review

In [1]:
from Bio import SeqIO, AlignIO
import math
import pandas as pd
import glob
import numpy as np
AAs = ['A','C','D','E','F','G','H','I','K','L','M','N','P','Q','R','S','T','V','W','Y']

---
## Shannon information:
### $$-\sum_{i=1}^{20} p_{i} log_{2}p_{i}$$ ###

#### or use: $$\sum_{i=1}^{20} p_{i} log_{2}\left( \frac{1}{p_{i}} \right)$$ ####

where for the $p$ fraction of the $i^{th}$ amino acid for a given site

check on "relative shannon entropy"


In [2]:
#take in a string of letters for a given site
#return Shannon information for that site

def shannon_info(site, AAs=AAs):
    
    temp_info = 0
    
    for aa in AAs:
        
        if aa in site:
            
            temp_info += site.count(aa)/len(site) * math.log2(site.count(aa)/len(site))
        
    return abs(temp_info)

need MC methods for confidence interval

---
## DIVAA:
### $$d= \frac{1}{N\sum_{k}p^2_{k}}$$ ###

where $N = 20$ for the 20 amino acids and $p_{k}$ is the fraction of the $k^{th}$ amino acid at a given site

In [3]:
#take in a string of letters for a given site
#return amino acid diversity for that site

#take in a dictionary of frequencies of each amino acid for a given site
def sigma(f_k, n_seqs):
        
    return math.sqrt((f_k * (1 - f_k)) / n_seqs)


def DIVAA(site, AAs=AAs):
    
    temp_divaa = 0
    
    for aa in AAs:
        
        if aa in site:
            
            temp_divaa += (site.count(aa)/len(site))**2
                
    return 1/(20 * temp_divaa)



def DIVe(site, AAs=AAs):
    
    temp_divaa = 0
    
    for aa in AAs:
            
        if aa in site:
            
            temp_divaa += (((site.count(aa)/len(site))**2) - sigma(site.count(aa)//len(site), len(site)))
    
    return 1 / (20 * temp_divaa)

## $\sigma(f_{k})$ ##

for use in estimation correction of d - <b> likely typos in the equations, double check paper </b>

$$\sigma(f_{k}) = \sqrt{ \frac{f_{k}(1 - f_{k})}{n_{seq}} }$$

where $f_{k}$ if the observed frequency of the $k^{th}$ amino acid at a given site. This can then be used to calculate an estimate of d that corrects for underestimation of diversity by using the observed frequencies, $f_{k}$, for $p_{k}$:
$$d_{e} = \frac{1}{20 \Pi_{k}(f_{k}^{2} - \sigma(f_{k}))}$$

___
## $K_{aa}$
$$K_{aa} = -ln(1-d_{aa})$$

where $d_{aa}$ is the proportion of sites differing between two protein sequences.

In [4]:
def Kaa(sample1,sample2):
    
    #takes in two aligned amino acid sequences, each as a string
    
    N = len(sample1)
    diffs = 0
           
    for site in range(max([len(sample1),len(sample2)])):
           
           if (sample1[site].lower() in AAs) & (sample2[site].lower() in AAs):
           
               N += 1
               
               if (sample1[site].lower() != sample2[site].lower()):
           
                    diffs +=1
    
    d_aa = diffs / N
        
    return (-log(1-daa), V_K_aa(d_aa,N)) #returns K_aa and V(K_aa) as a tuple

## $V(K_{aa})$
### $$V(K_{aa}) = \frac{d_{aa}}{N(1 - d_{aa})}$$ ###

where $N$ is the number of amino acids that can be compared between the two sequences (i.e. exlcuding any indels).

In [5]:
def V_K_aa(d_aa,N, AAs=AAs):
    
    #takes in d_aa (proportion of differences between two aa sequences)
    #    and N, number of comperable sites (excluding all indels)
    
    return d_aa / ((1 - d_aa) * N) #returns variance of K_aa

___
## Pairwise identity:
### $$\sum_{ij}x_{i}x_{j}\pi_{ij}$$ ###

where for the $x_{i}^{th} and x_{j}^{th}$ fractions of the $i$ and $j$ variants when $i\neq j$ and $\pi_{ij}$ is the number of differences between sequences for a given site, so
$$\pi_{ij} =\begin{cases} 0 & i=j \\
                             1 & i\neq j 
               \end{cases}$$

In [6]:
#take in a string of letters for a given site
#return amino acid diversity for that site

def pairwise_ident(site, AAs=AAs):

    ident = 0
    
    for i in range(len(AAs)-1):
        
        for j in range(i+1,len(AAs)):
                
            ident += (site.count(AAs[i])/len(site)) * (site.count(AAs[j])/len(site))
       
    return ident * 2

---
## Pairwise similarity:
### $$\sum_{ij}x_{i}x_{j}\pi_{ij}\delta_{ij}$$ ###

where $\delta_{ij}$ is added to the pairwise identity caculation and is the weighting for the $i \Leftrightarrow j$ substitution. For now, focus on symmetric substitution matrix or weighting(s).

In [7]:
#https://rutgers.zoom.us/j/92918120872?pwd=N0xCTXVlbWNLc1R0SS9IYmk1c0ZIZz09def pairwise_sim(site, subst_matrix, AAs=AAs):

#take in a string of letters for a given site
#takes in a substitution matrix as a Pandas DataFrame
#return amino acid diversity for that site

def PIaa(site,subst_matrix,AAs=AAs):

    similarity = 0
    
    for i in range(len(AAs)-1):
        
        for i in range(len(AAs)-1):

            for j in range(i+1,len(AAs)):   
                
                similarity += (site.count(AAs[i])/len(site)) * (site.count(AAs[j])/len(site)) * 1/subst_matrix.loc[AAs[i],AAs[j]]
        
    return similarity

## von Neumann entropy (VNE):

 From Caffrey et al 2004 (https://onlinelibrary.wiley.com/doi/full/10.1110/ps.03323604)

$$
    VNE = -Tr(\boldsymbol{\overline{\omega}} log_{20} \boldsymbol{\overline{\omega}} )
$$

where $\overline{\omega}$ is a density matrix with trace = 1 <br>
<br>
The density matrix is the product of relative frequencies of the amino acids in each alignment position and an appropriate similarity matrix (Caffrey et al. used BLOSUM 50, we want to use LG) so: <br>

$$
    \boldsymbol{\overline{\omega}} = diag(P(A), P(R), ...,P(V)) x \bf{BLOSUM50}
$$

Now VNE can be more efficiently calculated as:

$$
    VNE = -\sum{\lambda_{i}} log_{20}\lambda_{i}
$$

where $\lambda_i$ is the eigenvalues of $\boldsymbol{\overline{\omega}}$

### Except that this won't work with the LG matrix: 
BLOSUM50 satisfies the triangle equality for all triplets (see below) and LG diaganol values are defined so that the row sums are zero and therefore do not satisfy the triangle inequality. WAG, LG, HIV and FLU matrices are similar to LG.

BLOSUM50 is a symmetric similarity matrix with varying non-zero (positive)values along the diagonal

<br>


Sequence weights (Henikoff & Henikoff 1994) can be incorporated by:

$$
 Freq (aa_{i}) = \sum_{j}w_{j}/n
$$

where $w_{j}$ is the sequence weight of sequence $j$ in which amino acid $aa_{ij}$ is observed and $n$ is the number of sequences. The weights sum to $n$


In [13]:
def log_20(x):
    try:
        return math.log(x,20)
    except:
        return 0

vlog_20 = np.vectorize(log_20)


In [8]:
def vn_entropy(column,sub_matrix,matrix_label):

    column_diag = np.zeros((sub_matrix.shape[0],sub_matrix.shape[0]))
    
    for aa in matrix_label:
        column_diag[matrix_label.index(aa),matrix_label.index(aa)] = column.count(aa) / len(column)

    omega = column_diag * sub_matrix 
    
    entropy = - omega * vlog_20(omega)
    
    return entropy.trace()


---
# Preliminary proposal Aims and preliminary data

### Preliminary Data:

## To do:


<strike>Read caffrey et al 2004 and Henikoff & Henikoff 1994 <br>
Implement Caffrey et al 2004</strike> <br>
### Poster draft of diversity measures <br>


In [10]:
data_list = glob.glob("./HIV-1*aln.fas")

#AlignIO.read("./HIV-1_rev_mafft.fas",'fasta')


In [12]:
LG_aa_order = ['A','R','N','D','C','Q','E','G','H','I','L','K','M','F','P','S','T','W','Y','V']
B50_aa_order = ['A','R','N','D','C','Q','E','G','H','I','L','K','M','F','P','S','T','W','Y','V','B','J','Z','X','*']


In [83]:
#choose substitution matrix:

#original LG
LG = pd.read_csv("LG.csv",header=0, index_col=0)

#LG with bottom half (exclusive of the diagnal) proportions sum to 1
#Don't use this, only inflates values
#substitution_matrix = pd.read_csv("./protein_diversity/LG-1.csv",header=0, index_col=0)


In [10]:
B50 = pd.read_table("BLOSUM50.tab",header=None,index_col=None,names=B50_aa_order)
B50.set_index(pd.Series(B50_aa_order),inplace=True)

In [ ]:
max_freq = np.array([[0.05,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0],
                     [0,0.05,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0],
                     [0,0,0.05,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0],
                     [0,0,0,0.05,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0],
                     [0,0,0,0,0.05,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0],
                     [0,0,0,0,0,0.05,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0],
                     [0,0,0,0,0,0,0.05,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0],
                     [0,0,0,0,0,0,0,0.05,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0],
                     [0,0,0,0,0,0,0,0,0.05,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0],
                     [0,0,0,0,0,0,0,0,0,0.05,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0],
                     [0,0,0,0,0,0,0,0,0,0,0.05,0,0,0,0,0,0,0,0,0,0,0,0,0,0],
                     [0,0,0,0,0,0,0,0,0,0,0,0.05,0,0,0,0,0,0,0,0,0,0,0,0,0],
                     [0,0,0,0,0,0,0,0,0,0,0,0,0.05,0,0,0,0,0,0,0,0,0,0,0,0],
                     [0,0,0,0,0,0,0,0,0,0,0,0,0,0.05,0,0,0,0,0,0,0,0,0,0,0],
                     [0,0,0,0,0,0,0,0,0,0,0,0,0,0,0.05,0,0,0,0,0,0,0,0,0,0],
                     [0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0.05,0,0,0,0,0,0,0,0,0],
                     [0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0.05,0,0,0,0,0,0,0,0],
                     [0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0.05,0,0,0,0,0,0,0],
                     [0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0.05,0,0,0,0,0,0],
                     [0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0.05,0,0,0,0,0],
                     [0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0],
                     [0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0],
                     [0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0],
                     [0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0],
                     [0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0]])

In [ ]:
#VNE value at Shannon Entropy Max: all amino acids at frequency 0.05
omega = B50 * max_freq
ent = -(omega * vlog_20(omega))
ent.trace()

In [ ]:
#Pairwise similarity maximum for two allele locus (C,E)
(1/LG.loc['C','E']) * 0.5

In [ ]:
#VNE maximum at two allele locus (C,W)
max_vne = np.array([[0.090909,0],[0,0.07878788]])
omega = max_vne * np.array([[0.5,0],[0,0.5]])
ent = -(omega * vlog_20(omega))
ent.trace()

In [ ]:
#calculate the diversity measures
for currentfile in data_list:
    data = AlignIO.read(currentfile,'fasta')
    print("{} measures:".format(currentfile))    


    num_samples = len(data)
    ungapped_sites = 0

    shannon = 0
    Divaa = 0
    Divaa_e = 0
    p_ident = 0
    p_sim = 0
    vne = 0

    
    #get diversity meaasure(s) for ungapped sites
    for i in range(len(data[0,:])):
    
    
        column = data[:,i]
        if '-' not in column:
        
            shannon += shannon_info(column)
            Divaa += DIVAA(column)
            Divaa_e += DIVe(column)
            p_ident += pairwise_ident(column)
            p_sim += PIaa(column, LG)
            vne += vn_entropy(column,B50,B50_aa_order)
    
                
    print("shannon: {:.4}".format(shannon/(i+1)))
    print("DIVAA: {:.4}".format(Divaa/(i+1)))
    print("Divaa_est: {:.4}".format(Divaa_e/(i+1)))
    print("Pi Identity: {:.4}".format(p_ident/(i+1)))
    print("Pi similarity: {:.4}".format(p_sim/(i+1)))
    print("VNE: {:.4}".format(vne/(i+1)))
    
                
                
                

In [ ]:
def residue_weight_freq():
    #caculate sequence weighted frequencies as per Henikoff & Henikoff 1994
    
    """ for n sequences, r unique amino acids"""
    #for each alignment position
        
        #for each sequence
            #position weight = 1 / [(count of unique residues)* (count of residues found in this postion that are the same as the residue in this sequence at this position)
            
            #add to running total for this sequence
            
        
    #weights need to be normalized
        #to replicate Henikoff & Henikoff 1994, weights sum to 1
        #to replicate Caffrey et al 2004, weights sum to number of sequences
        
    
    return

def sequence_weight_freq():
    
    
    
    
    return


In [ ]:
print("shannon: {:.4}".format(shannon))
print("DIVAA: {:.4}".format(Divaa))
print("Pi Identity: {:.4}".format(p_ident))
print("Pi similarity: {:.4}".format(p_sim))

In [1]:
for i in range(1,21):
    print("{}\t{}".format(i,1 / (20 * ((1/i)**2)*i)))
    

1	0.05
2	0.1
3	0.15
4	0.2
5	0.24999999999999994
6	0.3
7	0.35000000000000003
8	0.4
9	0.44999999999999996
10	0.4999999999999999
11	0.5499999999999999
12	0.6
13	0.6499999999999999
14	0.7000000000000001
15	0.7499999999999999
16	0.8
17	0.85
18	0.8999999999999999
19	0.9500000000000001
20	0.9999999999999998


In [ ]:
for i in range(1,21):
    temp = []
    for j in range(i):
        temp += [LG_aa_order[j]] * i
    print(pairwise_ident(temp))
    

In [21]:
#normed B50 array for VNE calculation:
#    adjust diaganol to sum to 1
#    ignore off-diagonal as it's not used
B50_trace = np.array(B50).trace()
for aa in B50_aa_order:
    B50.loc[aa,aa] = B50.loc[aa,aa]/B50_trace


In [49]:
import itertools

In [59]:
#VNE theoretical max
B50_fast_first = ['W','C','H','P','D','G','F','Y','R','N','Q','M','E','K','A','I','L','S','T','B','J','Z','X','*']

for i in range(1,21):
    permutations = list(itertools.combinations(LG_aa_order,i))
    try:
        freq = 1/i
    
    VNEs = []
    
    for permutation in permutations:
        max_freqs = pd.DataFrame(0,index=B50.index,columns=B50.columns)
    
        for alleles in permutations:
            for allele in alleles:
                max_freqs.loc[allele,allele] = freq
    
        omega = np.array(max_freqs)*np.array(B50)
        VNEs.append(-(omega * vlog_20(omega)).trace())

    print(i,max(VNEs))

1 0.9206291797528904
2 0.5654865049490629
3 0.41800542093038295


KeyboardInterrupt: 

In [81]:

for i in range(1,21):
    permutations = list(itertools.combinations(LG_aa_order,i))
    
    freq = 1/i
    
    VNEs = {}
    
    for permutation in permutations:
        max_freqs = pd.DataFrame(0,index=B50.index,columns=B50.columns)
    
        for alleles in permutation:
            for allele in alleles:
                max_freqs.loc[allele,allele] = freq

        omega = np.array(max_freqs)*np.array(B50)
        VNEs[(-(omega * vlog_20(omega)).trace())] = permutation

    with open("VNE.txt", 'a') as f:
        f.write(str(i)+"\t"+str(max(VNEs))+"\t"+str(VNEs[max(VNEs)])+"\n")
    print("Done with {}".format(i))

Done with 1
Done with 2
Done with 3
Done with 4
Done with 5
Done with 6
Done with 7
Done with 8
Done with 9
Done with 10
Done with 11
Done with 12
Done with 13
Done with 14
Done with 15
Done with 16
Done with 17
Done with 18
Done with 19
Done with 20


In [79]:
for i in range(1,21):
    permutations = list(itertools.combinations(LG_aa_order,i))
    print(i, len(permutations))

1 20
2 190
3 1140
4 4845
5 15504
6 38760
7 77520
8 125970
9 167960
10 184756
11 167960
12 125970
13 77520
14 38760
15 15504
16 4845
17 1140
18 190
19 20
20 1


In [ ]:
for i in range(1,21):
    print("{}\t{}".format(i,(-((1/i))*log_20(1/i))*i))
    

## Aim 1: Develop an evolvability measure based on amino acid diversity that can be used in comparisons between species.
##  Aim 2: Test against Shannon Information and DIVAA amino acid diversity measures using influenza A (-ssRNA), HIV(retrovirus), and rotavirus(dsRNA) protein (8-10 proteins ea.) alignments.


#### Aim 1

Background and Utility of Shannon Information and DIVAA measures.<br>
1.Useful scope of amino acid information in relation to nucleotide data
2.Useful scope of SI and DIVAA
3.Limitations of SI and DIVAA

#### Aim 2
Background and Utility of Amino Acid Substitution Matrices.<br>
1.Useful scope of AA substitution matrices in relation to nuc mutation rate models
2. 

TODO: preliminary data-run SI and DIVAA (and Kaa?) on 1 or 2 viral proteins
goto dryad and zenoto to find viral protein alignments

Flu matrix, HIV matrix

Specific Aim 1: Develop evolvability metric based on amino acid diversity that can be used in comparisons between species
a. A base model of pairwise similarity excluding any specific model of protein evolution
b. Incorporating the LG model of protein evolution

Specific Aim 2: Test novel metric against existing measures using a set of viral protein alignments (Influenza A, HIV, rotavirus).

### Check Triangle Inequality for LG matrix ###

In [144]:
#load modified LG matrix to satisfy Baussand & Carbone 2008 assumptions, methods
LG_full = pd.read_csv("LG_full.csv",header=0, index_col=0)

In [18]:
ap =  ['A','C','D','E','F','G','H','I','K','L','M','N','Q','R','S','T','V','W','Y']
hy = ['A','C','D','E','G','H','K','N','P','Q','R','S','T']

In [157]:
def check_triangle(aa_set,matrix):
    
    def distance(i,j,df):
        return abs(df.loc[i,i] - df.loc[i,j])
    
    count = 0
    fail = 0
    for x in aa_set:
        for y in aa_set:
            for z in aa_set:
                if (x != z) & (y != z):
                    count += 1
                    if distance(x,y,matrix) > (distance(x,z,matrix) + distance(z,y,matrix)):
                        if (distance(x,y,matrix) - (distance(x,z,matrix) + distance(z,y,matrix))) > 1:
                            print("triplet {}, {}, {} fails at {}!".format(x,y,z, (distance(x,y,matrix) - (distance(x,z,matrix) + distance(z,y,matrix)))))
                            
                            fail += 1
                    
    print("{} of {} triplets failed!".format(fail,count))
    return
    """
    count = 0
    fail = 0
    for x in aa_set:
        for y in aa_set:
            for z in aa_set:
                if (x != z) & (y != z):
                    count += 1
                    if abs(matrix.loc[x,y]) > abs(matrix.loc[x,z]) + abs(matrix.loc[z,y]):
                        if (matrix.loc[x,y] - (matrix.loc[x,z] + matrix.loc[z,y])) > 1:
                            #print("triplet {}, {}, {} fails at {}!".format(x,y,z, matrix.loc[x,y] - (matrix.loc[x,z] + matrix.loc[z,y])))
                            fail += 1
                    
    print("{} of {} triplets failed!".format(fail,count))
    return"""                

In [158]:
check_triangle(AAs,LG_full)

triplet A, I, V fails at 2.3980400000000004!
triplet A, N, G fails at 1.1608270000000003!
triplet A, N, S fails at 1.9864430000000004!
triplet A, T, S fails at 2.5876810000000003!
triplet C, I, V fails at 1.638664!
triplet C, T, S fails at 1.6409980000000002!
triplet D, H, N fails at 3.5821240000000003!
triplet D, K, E fails at 1.5242180000000003!
triplet D, K, N fails at 1.8621189999999999!
triplet D, Q, E fails at 3.6052049999999998!
triplet D, Q, N fails at 1.1723659999999998!
triplet D, S, N fails at 2.768083!
triplet D, T, N fails at 1.5748189999999997!
triplet E, H, Q fails at 3.7047100000000004!
triplet E, N, D fails at 4.534437!
triplet E, R, K fails at 1.4432070000000001!
triplet E, R, Q fails at 1.7590240000000001!
triplet F, H, Y fails at 4.624694999999999!
triplet G, T, S fails at 1.610154!
triplet H, D, N fails at 3.5821239999999994!
triplet H, E, Q fails at 3.7047100000000004!
triplet H, F, Y fails at 4.624694999999999!
triplet H, K, Q fails at 2.53703!
triplet H, K, R fa

In [159]:
check_triangle(ap,LG_full)

triplet A, I, V fails at 2.3980400000000004!
triplet A, N, G fails at 1.1608270000000003!
triplet A, N, S fails at 1.9864430000000004!
triplet A, T, S fails at 2.5876810000000003!
triplet C, I, V fails at 1.638664!
triplet C, T, S fails at 1.6409980000000002!
triplet D, H, N fails at 3.5821240000000003!
triplet D, K, E fails at 1.5242180000000003!
triplet D, K, N fails at 1.8621189999999999!
triplet D, Q, E fails at 3.6052049999999998!
triplet D, Q, N fails at 1.1723659999999998!
triplet D, S, N fails at 2.768083!
triplet D, T, N fails at 1.5748189999999997!
triplet E, H, Q fails at 3.7047100000000004!
triplet E, N, D fails at 4.534437!
triplet E, R, K fails at 1.4432070000000001!
triplet E, R, Q fails at 1.7590240000000001!
triplet F, H, Y fails at 4.624694999999999!
triplet G, T, S fails at 1.610154!
triplet H, D, N fails at 3.5821239999999994!
triplet H, E, Q fails at 3.7047100000000004!
triplet H, F, Y fails at 4.624694999999999!
triplet H, K, Q fails at 2.53703!
triplet H, K, R fa

In [160]:
check_triangle(hy,LG_full)

triplet A, N, G fails at 1.1608270000000003!
triplet A, N, S fails at 1.9864430000000004!
triplet A, T, S fails at 2.5876810000000003!
triplet C, T, S fails at 1.6409980000000002!
triplet D, H, N fails at 3.5821240000000003!
triplet D, K, E fails at 1.5242180000000003!
triplet D, K, N fails at 1.8621189999999999!
triplet D, Q, E fails at 3.6052049999999998!
triplet D, Q, N fails at 1.1723659999999998!
triplet D, S, N fails at 2.768083!
triplet D, T, N fails at 1.5748189999999997!
triplet E, H, Q fails at 3.7047100000000004!
triplet E, N, D fails at 4.534437!
triplet E, R, K fails at 1.4432070000000001!
triplet E, R, Q fails at 1.7590240000000001!
triplet G, T, S fails at 1.610154!
triplet H, D, N fails at 3.5821239999999994!
triplet H, E, Q fails at 3.7047100000000004!
triplet H, K, Q fails at 2.53703!
triplet H, K, R fails at 1.7293370000000001!
triplet H, S, N fails at 2.451435!
triplet K, D, E fails at 1.5242180000000003!
triplet K, D, N fails at 1.8621189999999999!
triplet K, H, Q 

In [149]:
check_triangle(AAs,B50)

0 of 7220 triplets failed!


In [122]:
check_triangle(hy,B50)

0 of 1872 triplets failed!
